In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4 #how many blocks in parallel
learning_rate = 3e-4
#eval_interval = 2500
max_iters = 1000
eval_iters = 250
#dropout = 0.2 #dropout random neurons of network = don't overfit

cuda


In [2]:
with open('wizard-of-oz.txt', 'r', encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s];
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    #print(ix)
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device) #pushed to GPU = will be handled there
    return x,y

x, y = get_batch('train')
print("inputs: ")
print(x)

print ("targets: ")
print(y)
    

inputs: 
tensor([[57,  1, 72, 68,  1, 58, 72, 56],
        [ 1, 67, 68, 73,  1, 75, 58, 60],
        [71, 72, 58,  9,  1, 61, 54, 67],
        [58, 57,  1, 56, 54, 74, 73, 62]], device='cuda:0')
targets: 
tensor([[ 1, 72, 68,  1, 58, 72, 56, 54],
        [67, 68, 73,  1, 75, 58, 60, 58],
        [72, 58,  9,  1, 61, 54, 67, 60],
        [57,  1, 56, 54, 74, 73, 62, 68]], device='cuda:0')


In [5]:

x = train_data[:block_size]
y = train_data[1:block_size + 1]

print(x)
print(y)

for t in range(block_size):
    context = x[:t+1]
    target=y[t]
    print("when input is", context, "target is", target)

tensor([80,  1,  1, 28, 39, 42, 39, 44])
tensor([ 1,  1, 28, 39, 42, 39, 44, 32])
when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [6]:
#decorator makes pytorch not use gradients at all in it
# = reduces computation / memory usage
#NO TRAINING IS HAPPENING HERE
@torch.no_grad()
def estimate_loss():
    out = {}

    # puts model into eval mode - for evaluating / testing
    # ** stuff disabled
    # want to see how well model actually performs
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
        
        # puts model into training mode
        # ** weights + biases update during this phase
        # ** dropout + batch normalisation active
        model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            # batch size, ('time steps' or 'sequence length'), 'channels' = dimensionality of vectors =  vocab_size  
            B, T, C = logits.shape

            #flatten, now where the characters are in their individual sequences doesn't matter
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)

            #compares the vectors from embedding table to the truth value (truth vectors automatically generated, pytorch expects only the actual truth val here)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    #we're not training here, just using existing model
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            
            # focus only on the last time step AS ONLY WANT NEXT CHARACTER
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device) #push to GPU

context = torch.zeros((1,1), dtype=torch.long, device=device) #nextline character
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


fAoe1[,,LRZP]HAW﻿fzfHhs!hWh.N'pcrcYaeTAt)ezSvMH ueujuBm6y["p3hq0jF2McP&?fHi4bYe(BUcC
.P'&TrfxeJLG9G;9b?KI31BRg7NQGA﻿"7]4vgz_.E﻿vE,fYG82;aoeaQOk8OaEPvgUyRpy[zAU9SJ?6'hK:'hnNr9u86g1YE-][m'"AB8b5hiCBB.TKb.)*Vmb3﻿5554bV(0bkZRq;X5ZEPOFAWnw3c-e0Jy*u]HzUM(nL&xddL﻿"MKI"QtUn90J&"QNT1_dfMaJ,Ty[ypC"xx
6yRg3cdNSvb9j3!(7[)﻿"HhzD(6ZW8chk2n;﻿xF?z_﻿cwV:9DI:L'
r E0MN:oVqn;)eKQ!(sNy6BCl﻿86Zg*klfSqrXADZ'04Dn﻿30vm'nH6CK;QN6cuoPY(sN'"WbW5eJ﻿D0M)*R7GfL&]I!vplu-X-DzuT_W*1.PC;7)95l!n(4j&zoEP.P]ggg9vM8W3BewcVWi'Uj!A*FER


In [8]:
a = torch.rand(2, 3, 5)
x, y, z = a.shape
a = a.view(x, y, z)
a.shape

torch.Size([2, 3, 5])

In [9]:
#pytorch optimizer:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_inters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    #sample a batch of data
    xb, yb = get_batch('train') #xb = index / inputs, yb = target

    #evaluate the loss
    logits, loss = model.forward(xb, yb)

    #so previous gradients don't affect current gradient
    #previous gradients are from previous data
    #none instead of 0 as optimisation for less space (0 would be stored as int64)
    optimizer.zero_grad(set_to_none=True)

    #computes gradients
    loss.backward()

    #updates model's weights based on gradients
    optimizer.step()

print(loss.item())

NameError: name 'eval_inters' is not defined

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

In [ ]:
x = torch.tensor([-0.05], dtype=torch.float32)
y = F.sigmoid(x)
print(y)